In [2]:
import pandas as pd
import numpy as np

try:
    # Read the CSV file with latin1 encoding
    df = pd.read_csv('The Ultimate Cars Dataset 2024.csv', encoding='latin1')

    # Replace 'N/A' and 'N' with NaN
    df = df.replace(['N/A', 'N'], np.nan)

    # Clean price data - handle ranges by taking the first value
    df['Cars Prices'] = df['Cars Prices'].str.replace('$', '').str.replace(',', '')
    df['Cars Prices'] = df['Cars Prices'].apply(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)
    df['Cars Prices'] = df['Cars Prices'].apply(lambda x: x.split('-')[0] if isinstance(x, str) and '-' in x else x)
    df['Cars Prices'] = pd.to_numeric(df['Cars Prices'].str.strip(), errors='coerce')

    # Clean horsepower data
    df['HorsePower'] = df['HorsePower'].str.replace(' hp', '')
    df['HorsePower'] = df['HorsePower'].apply(lambda x: x.split('-')[0] if isinstance(x, str) and '-' in x else x)
    df['HorsePower'] = pd.to_numeric(df['HorsePower'].str.strip(), errors='coerce')

    # Clean performance data
    df['Performance(0 - 100 )KM/H'] = df['Performance(0 - 100 )KM/H'].str.replace(' sec', '')
    df['Performance(0 - 100 )KM/H'] = df['Performance(0 - 100 )KM/H'].apply(lambda x: x.split('-')[0] if isinstance(x, str) and '-' in x else x)
    df['Performance(0 - 100 )KM/H'] = pd.to_numeric(df['Performance(0 - 100 )KM/H'].str.strip(), errors='coerce')

    # Clean speed data
    df['Total Speed'] = df['Total Speed'].str.replace(' km/h', '')
    df['Total Speed'] = pd.to_numeric(df['Total Speed'].str.strip(), errors='coerce')

    # Additional data cleaning steps
    print("\nInitial Data Shape:", df.shape)
    print("\nMissing Values Before Cleaning:")
    print(df.isnull().sum())

    # Handle missing values
    # For numerical columns, fill with median
    numerical_columns = ['HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices']
    for col in numerical_columns:
        df[col] = df[col].fillna(df[col].median())

    # For categorical columns, fill with mode
    categorical_columns = ['Company Names', 'Cars Names', 'Fuel Types']
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Remove duplicates
    df = df.drop_duplicates()

    # Handle outliers using IQR method
    for col in numerical_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower_bound, upper_bound)

    # Standardize text columns
    df['Company Names'] = df['Company Names'].str.strip().str.title()
    df['Cars Names'] = df['Cars Names'].str.strip().str.title()
    df['Fuel Types'] = df['Fuel Types'].str.strip().str.title()

    print("\nMissing Values After Cleaning:")
    print(df.isnull().sum())
    print("\nFinal Data Shape:", df.shape)

    # Basic statistics for numerical columns
    print("\nBasic Statistics for Numerical Columns:")
    stats = df[['HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices']].describe()
    print(stats.round(2))

    # Group by Company Names and calculate mean statistics
    print("\nTop 10 Companies by Average Horsepower:")
    company_stats = df.groupby('Company Names').agg({
        'HorsePower': 'mean',
        'Total Speed': 'mean',
        'Performance(0 - 100 )KM/H': 'mean',
        'Cars Prices': 'mean'
    }).round(2)

    print(company_stats.sort_values(by='HorsePower', ascending=False).head(10))

    # Group by Fuel Types and calculate mean statistics
    print("\nMean Statistics by Fuel Type:")
    fuel_stats = df.groupby('Fuel Types').agg({
        'HorsePower': 'mean',
        'Total Speed': 'mean',
        'Performance(0 - 100 )KM/H': 'mean',
        'Cars Prices': 'mean'
    }).round(2)

    print(fuel_stats)

    # Calculate correlation between numerical columns
    print("\nCorrelation between numerical columns:")
    correlation = df[['HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices']].corr()
    print(correlation.round(2))

    # Additional analysis
    print("\nTop 5 Most Expensive Cars:")
    print(df.nlargest(5, 'Cars Prices')[['Company Names', 'Cars Names', 'Cars Prices', 'HorsePower']])

    print("\nTop 5 Fastest Cars (0-100 km/h):")
    print(df.nsmallest(5, 'Performance(0 - 100 )KM/H')[['Company Names', 'Cars Names', 'Performance(0 - 100 )KM/H', 'HorsePower']])

except Exception as e:
    print(f"An error occurred: {str(e)}") 


Initial Data Shape: (1213, 11)

Missing Values Before Cleaning:
Company Names                  0
Cars Names                     0
Engines                        0
CC/Battery Capacity            3
HorsePower                   186
Total Speed                    1
Performance(0 - 100 )KM/H     10
Cars Prices                    1
Fuel Types                     0
Seats                          0
Torque                         1
dtype: int64

Missing Values After Cleaning:
Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    0
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

Final Data Shape: (1208, 11)

Basic Statistics for Numerical Columns:
       HorsePower  Total Speed  Performance(0 - 100 )KM/H  Cars Prices
count     1208.00      1208.00 